In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [2]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Search Tool

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=50,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

# Create ToolNode

In [5]:
from langgraph.prebuilt import ToolNode
from spotify_tools import get_playlists, create_spotify_playlist, get_track_list_from_playlist, add_tracks_to_playlist, filter_artists, get_artists_from_playlist, find_similar_artists, find_top_tracks
from plan import validate_plan

tools = [get_playlists, create_spotify_playlist, add_tracks_to_playlist, filter_artists, validate_plan, get_artists_from_playlist, search_tool, find_similar_artists, get_track_list_from_playlist, find_top_tracks]
# tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, search_tool]
tool_node = ToolNode(tools)

# Bind Tools to Model

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=1.0)
# llm_with_tools = llm.bind_tools(tools, strict=True, parallel_tool_calls=False)
llm_with_tools = llm.bind_tools(tools, strict=True)

# First Message

In [7]:
from langchain_core.messages import HumanMessage
from prompts import Prompts

human_message = HumanMessage(Prompts.SPOTIFY)
ai_message = llm_with_tools.invoke([human_message])

2024-11-13 21:38:20,972 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [8]:
if ai_message.tool_calls is None or ai_message.tool_calls[0]["name"] != "validate_plan":
    raise ValueError("should be tool message calling 'validate_plan'")

In [9]:
ai_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  validate_plan (call_MPPlKioQb7RXTUePSlYBwe0s)
 Call ID: call_MPPlKioQb7RXTUePSlYBwe0s
  Args:
    plan: {'steps': [{'name': 'retrieve_artists_from_existing_playlist', 'description': "Get the list of unique artists from the existing 'New Rock and Blues' playlist.", 'success_criteria': "Successfully retrieve a list of unique artist URIs from the 'New Rock and Blues' playlist.", 'tool': 'functions.get_artists_from_playlist', 'action': "Retrieve the unique artist URIs from the 'New Rock and Blues' playlist."}, {'name': 'find_similar_artists', 'description': "For each artist in the 'New Rock and Blues' playlist, find 3-4 artists with a similar music style.", 'success_criteria': 'A comprehensive list of similar artists is obtained for each artist in the original playlist.', 'tool': 'functions.find_similar_artists', 'action': 'Using the retrieved artist URIs, find similar artists for each.'}, {'name

In [10]:
plan = ai_message.tool_calls[0]["args"]
plan

{'plan': {'steps': [{'name': 'retrieve_artists_from_existing_playlist',
    'description': "Get the list of unique artists from the existing 'New Rock and Blues' playlist.",
    'success_criteria': "Successfully retrieve a list of unique artist URIs from the 'New Rock and Blues' playlist.",
    'tool': 'functions.get_artists_from_playlist',
    'action': "Retrieve the unique artist URIs from the 'New Rock and Blues' playlist."},
   {'name': 'find_similar_artists',
    'description': "For each artist in the 'New Rock and Blues' playlist, find 3-4 artists with a similar music style.",
    'success_criteria': 'A comprehensive list of similar artists is obtained for each artist in the original playlist.',
    'tool': 'functions.find_similar_artists',
    'action': 'Using the retrieved artist URIs, find similar artists for each.'},
   {'name': 'filter_existing_artists',
    'description': "Remove artists from the new artist list that are already present in the 'New Rock and Blues' playlist.

# chat_prompt_template holds all messages (Human, AI, Tool)

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt_template: ChatPromptTemplate = human_message + ai_message
messages = chat_prompt_template.format_messages()
messages

[HumanMessage(content="\n    Create, validate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n    **Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' \n        playlist, following the rules below.\n\n    **Rules:**\n\n    1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n    2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n    3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n    4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in \n        the 'New Rock and Blues' playlist.\n\n    5. **Recommend 3-4 tracks for each new artist.\n\n    6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, \n        consider

# Check for Tools in Message

In [ ]:
from langchain_core.messages import ToolMessage

tools_in_ai_message = []
tools_by_name = {tool.name: tool for tool in tools}
messages = chat_prompt_template.format_messages()
for tool_call in messages[-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    tools_in_ai_message.append(tool)
    # observation = tool.invoke(tool_call["args"])
    # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
if len(tools_in_ai_message) == 0:
    raise ValueError
else:
    print(tools_in_ai_message)

# Tool Call (should be validate_plan())

In [12]:
response = tool_node.invoke({"messages": messages})
tool_message = response["messages"][0]
tool_message.pretty_print()

================================= Tool Message =================================
Name: validate_plan

true


# Add Tool Response to Messages

In [13]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

[HumanMessage(content="\n    Create, validate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n    **Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' \n        playlist, following the rules below.\n\n    **Rules:**\n\n    1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n    2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n    3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n    4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in \n        the 'New Rock and Blues' playlist.\n\n    5. **Recommend 3-4 tracks for each new artist.\n\n    6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, \n        consider

# Send Messages with Tool result to LLM

In [14]:
ai_tool_call_message = llm_with_tools.invoke(chat_prompt_template.format_messages())
ai_tool_call_message.pretty_print()

2024-11-13 21:42:13,848 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

The plan has been successfully validated. Let's proceed with the first step.

### Step 1: Retrieve Artists from Existing Playlist

**Objective:** Get the list of unique artists from the existing 'New Rock and Blues' playlist.

I will now retrieve the unique artist URIs from the 'New Rock and Blues' playlist.
Tool Calls:
  get_playlists (call_I92PMIWLcDF0TpGjrU0dHgCy)
 Call ID: call_I92PMIWLcDF0TpGjrU0dHgCy
  Args:


# Validate correct tool call

In [15]:
if ai_tool_call_message.tool_calls is None or ai_tool_call_message.tool_calls[0]["name"] != "get_playlists":
    raise ValueError("should be tool message calling 'validate_plan'")

# Tool Call (should be get_playlists())

In [16]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()
response = tool_node.invoke({"messages": messages})

In [17]:
from langchain_core.messages import ToolMessage

playlist_tool_message: ToolMessage = response["messages"][0]
playlist_tool_message.pretty_print()

================================= Tool Message =================================
Name: get_playlists

{"playlists": [{"id": "2dbYK5b7J0F7IdH5n1TEUK", "uri": "spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK", "name": "RPreacher", "description": "", "owner": "Ray", "tracks_total": 3, "is_public": true, "collaborative": false, "snapshot_id": "AAAABG1lhIiRJT/NE9RpP4K6UnmZWixo"}, {"id": "4ack9YtUhdxRayJDAqlfQe", "uri": "spotify:playlist:4ack9YtUhdxRayJDAqlfQe", "name": "RP Bossa Nova Chill ", "description": "", "owner": "Ray", "tracks_total": 30, "is_public": true, "collaborative": false, "snapshot_id": "AAAAIThdM9g/YPQwj7nuoHhewWyLoKEi"}, {"id": "75NW18NgdZuZeifrcjxKlZ", "uri": "spotify:playlist:75NW18NgdZuZeifrcjxKlZ", "name": "GVF", "description": "", "owner": "Ray", "tracks_total": 11, "is_public": true, "collaborative": false, "snapshot_id": "AAAADKZNV9h+cfi1PbOi6yjNHMe8mLWC"}, {"id": "5iyONtUO21O88xw8pBblwh", "uri": "spotify:playlist:5iyONtUO21O88xw8pBblwh", "name": "Now And Then", "descripti

# Add Tool Response to Messages

In [18]:
chat_prompt_template += playlist_tool_message
messages = chat_prompt_template.format_messages()

# Send Playlists Results to LLM

In [19]:
ai_tool_call_message = llm_with_tools.invoke(messages)
ai_tool_call_message.pretty_print()

2024-11-13 21:44:24,714 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_artists_from_playlist (call_LLMxWbuDZYLZWmcQPBqVxJIM)
 Call ID: call_LLMxWbuDZYLZWmcQPBqVxJIM
  Args:
    playlist_id: spotify:playlist:01tk0aitEuGK0ajWCkzdKc


# Validate correct tool call

In [20]:
if ai_tool_call_message.tool_calls is None or ai_tool_call_message.tool_calls[0]["name"] != "get_artists_from_playlist":
    raise ValueError("should be tool message calling 'validate_plan'")

# Add AI Tool Call to Messages

In [21]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()

# Tool Call (get_artists_from_playlist())

In [22]:
response = tool_node.invoke({"messages": messages})
track_list_tool_message: ToolMessage = response["messages"][0]

# Decode AI Tool Call message

In [23]:
track_list_tool_message.pretty_print()

================================= Tool Message =================================
Name: get_artists_from_playlist

{"artists": {"Gary Clark Jr.": "spotify:artist:01aC2ikO4Xgb2LUpf9JfKp", "Ayron Jones": "spotify:artist:1iEaqWaYpKo9x0OrEq7Q7z", "Thirty Seconds To Mars": "spotify:artist:0RqtSIYZmd4fiBKVFqyIqD", "KALEO": "spotify:artist:7jdFEYD2LTYjfwxOdlVjmc", "Arctic Monkeys": "spotify:artist:7Ln80lUS6He07XvHI8qqHH", "Queens of the Stone Age": "spotify:artist:4pejUc4iciQfgdX6OKulQn", "Black Pistol Fire": "spotify:artist:0Nrwy16xCPXG8AwkMbcVvo", "Welshly Arms": "spotify:artist:1xKrH6GSh9CJh8nYwbqW7B", "The Record Company": "spotify:artist:6vYg01ZFt1nREsUDMDPUYX", "The Heavy": "spotify:artist:0bZCak2tcRMY1dzEIuwF42", "Greta Van Fleet": "spotify:artist:4NpFxQe2UvRCAjto3JqlSl", "Whiskey Myers": "spotify:artist:26opZSJcXshCmCwxgZQmBc", "Colter Wall": "spotify:artist:3xYXYzm9H3RzyQgBrYwIcx", "Tyler Childers": "spotify:artist:13ZEDW6vyBF12HYcZRr4EV", "Chris Stapleton": "spotify:artist:4YLtscXsxb

In [ ]:
# json.loads(track_list_tool_message.content)

# Add Tool Response to Messages

In [24]:
chat_prompt_template += track_list_tool_message
messages = chat_prompt_template.format_messages()
messages

[HumanMessage(content="\n    Create, validate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n    **Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' \n        playlist, following the rules below.\n\n    **Rules:**\n\n    1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n    2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n    3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n    4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in \n        the 'New Rock and Blues' playlist.\n\n    5. **Recommend 3-4 tracks for each new artist.\n\n    6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, \n        consider

# Send Tool Result to LLM

In [25]:
response = llm_with_tools.invoke(messages)

2024-11-13 21:51:34,038 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI Tool Call (find_similar_artist())

In [ ]:
if ai_tool_call_message.tool_calls is None or ai_tool_call_message.tool_calls[0]["name"] != "get_artists_from_playlist":
    raise ValueError("should be tool message calling 'validate_plan'")

In [26]:
response.pretty_print()

================================== Ai Message ==================================

I have successfully retrieved the list of unique artists from the 'New Rock and Blues' playlist.

### Step 2: Find Similar Artists

**Objective:** For each artist in the 'New Rock and Blues' playlist, find 3-4 artists with a similar music style.

I will now find similar artists for each of the artists retrieved from the playlist. This might take several calls depending on the retrieved artist list. 

Let's proceed with finding similar artists.
Tool Calls:
  find_similar_artists (call_6XFb1sr4fV053QVbr8fZxlBf)
 Call ID: call_6XFb1sr4fV053QVbr8fZxlBf
  Args:
    artists: ['spotify:artist:01aC2ikO4Xgb2LUpf9JfKp', 'spotify:artist:1iEaqWaYpKo9x0OrEq7Q7z', 'spotify:artist:0RqtSIYZmd4fiBKVFqyIqD', 'spotify:artist:7jdFEYD2LTYjfwxOdlVjmc', 'spotify:artist:7Ln80lUS6He07XvHI8qqHH', 'spotify:artist:4pejUc4iciQfgdX6OKulQn', 'spotify:artist:0Nrwy16xCPXG8AwkMbcVvo', 'spotify:artist:1xKrH6GSh9CJh8nYwbqW7B', 'spotify:arti

In [28]:
if ai_tool_call_message.tool_calls is None or ai_tool_call_message.tool_calls[0]["name"] != "find_similar_artists":
    raise ValueError("should be tool message calling 'validate_plan'")

ValueError: should be tool message calling 'validate_plan'

# Add AI Tool Call to Messages

In [27]:
chat_prompt_template += response
messages = chat_prompt_template.format_messages()
messages

[HumanMessage(content="\n    Create, validate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n    **Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' \n        playlist, following the rules below.\n\n    **Rules:**\n\n    1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n    2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n    3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n    4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in \n        the 'New Rock and Blues' playlist.\n\n    5. **Recommend 3-4 tracks for each new artist.\n\n    6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, \n        consider

# Tool Call (find_similar_artists())

In [29]:
response = tool_node.invoke({"messages": messages})

In [30]:
tool_message = response["messages"][0]
tool_message.pretty_print()

================================= Tool Message =================================
Name: find_similar_artists

{'spotify:artist:1QAJqy2dA3ihHBFIHRphZj', 'spotify:artist:2yEwvVSSSUkcLeSTNyHKh8', 'spotify:artist:5krkohEVJYw0qoB5VWwxaC', 'spotify:artist:7MhMgCo0Bl0Kukl93PZbYS', 'spotify:artist:1Jl8u1U1GtBlwocZK5LWZv', 'spotify:artist:3Ayl7mCk0nScecqOzvNp6s', 'spotify:artist:3vbKDsSS70ZX9D2OcvbZmS', 'spotify:artist:6kbzJ40luXJ5IKnSpWr7SD', 'spotify:artist:241MOvr69jR2U48FW5WbKv', 'spotify:artist:7fJYw1vK9yWb8o51I8qHin', 'spotify:artist:48vfT7i3EmuNEDAVRhzxpp', 'spotify:artist:77SW9BnxLY8rJ0RciFqkHh', 'spotify:artist:521Jw1ixgckc0WJHhm2Klu', 'spotify:artist:3P5NW1wQjcWpR0VsT1m0xr', 'spotify:artist:74YaL1CwdKlao06KrJyuX2', 'spotify:artist:0epOFNiUfyON9EYx7Tpr6V', 'spotify:artist:53XhwfbYqKCa1cC15pYq2q', 'spotify:artist:3JtduJLfvJEAjll8oupFy5', 'spotify:artist:6u5mhJXgAKPTj6YVlZSPY9', 'spotify:artist:04HjibunL2jGRLEqVEMfZY', 'spotify:artist:4Dp0vVy8BpziagOylB21Tp', 'spotify:artist:2IXEnz2ferXu3

# Add Tool Response to Messages

In [31]:
chat_prompt_template += tool_message
messages = chat_prompt_template.format_messages()
messages

[HumanMessage(content="\n    Create, validate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n    **Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' \n        playlist, following the rules below.\n\n    **Rules:**\n\n    1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n    2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n    3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n    4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in \n        the 'New Rock and Blues' playlist.\n\n    5. **Recommend 3-4 tracks for each new artist.\n\n    6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, \n        consider

# Send Tool Result to LLM

In [32]:
response = llm_with_tools.invoke(messages)
response.pretty_print()

2024-11-13 22:27:28,621 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

I have identified a list of similar artists for the existing artists from the 'New Rock and Blues' playlist.

### Step 3: Filter Existing Artists

**Objective:** Remove artists from the new artist list that are already present in the 'New Rock and Blues' playlist.

I will now compare the list of retrieved similar artists with the existing playlist artists and filter out any duplicates.
Tool Calls:
  filter_artists (call_sCO0UKfk8rYDJxANjTjHhf3U)
 Call ID: call_sCO0UKfk8rYDJxANjTjHhf3U
  Args:
    playlist_id: spotify:playlist:01tk0aitEuGK0ajWCkzdKc
    new_artists: ['spotify:artist:1QAJqy2dA3ihHBFIHRphZj', 'spotify:artist:2yEwvVSSSUkcLeSTNyHKh8', 'spotify:artist:5krkohEVJYw0qoB5VWwxaC', 'spotify:artist:7MhMgCo0Bl0Kukl93PZbYS', 'spotify:artist:1Jl8u1U1GtBlwocZK5LWZv', 'spotify:artist:3Ayl7mCk0nScecqOzvNp6s', 'spotify:artist:3vbKDsSS70ZX9D2OcvbZmS', 'spotify:artist:6kbzJ40luXJ5IKnSpWr7SD', 'spotify:artist:2

In [33]:
chat_prompt_template += response
messages = chat_prompt_template.format_messages()
messages

[HumanMessage(content="\n    Create, validate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n    **Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' \n        playlist, following the rules below.\n\n    **Rules:**\n\n    1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n    2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n    3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n    4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in \n        the 'New Rock and Blues' playlist.\n\n    5. **Recommend 3-4 tracks for each new artist.\n\n    6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, \n        consider

In [37]:
hm = HumanMessage("Why did you only asked for similar artists for a ubset of all artists in the playlist? How can the prompt be improved to make sure you go through all of them?")
chat_prompt_template += hm
response = llm_with_tools.invoke(messages)
response.pretty_print()

2024-11-14 06:27:23,156 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_sCO0UKfk8rYDJxANjTjHhf3U", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

In [36]:
filtered_artists: ToolMessage = response["messages"][0]
filtered_artists.pretty_print()

================================= Tool Message =================================
Name: filter_artists

Error: TypeError("unsupported operand type(s) for -: 'set' and 'dict'")
 Please fix your mistakes.


In [ ]:
response = llm_with_tools.invoke(messages)

In [ ]:
chat_prompt_template += response
chat_prompt_template.format_messages()

In [ ]:
response = tool_node.invoke({"messages": [response]})

In [ ]:
response